In [1]:
# Importando as bibliotecas necessárias

from bs4 import BeautifulSoup     # Analisar e extrair dados de documentos HTML e XML
import requests                   # Realizar tarefas relacionadas a solicitações HTTP
import pandas as pd
from tqdm import tqdm             # Permite criar barras de progresso

In [2]:
class MercadoLivre():
    
    def __init__(self):

        self.site = 'https://www.mercadolivre.com.br/mais-vendidos#menu-user'
        self.gp_produtos = []
        self.link_produtos = []
        self.columns = ['categoria', 'titulo', 'preco', 'link']
        self.df_list = []
    
    
    def run(self):
        '''
            Processo para rodar as tarefas passo a passo
                Return 
                    DataFrame com todos os itens mais vendidos no site da ML, com 
                    as respectivas categoria e preços de cada item.
        '''
        
        self.get_mais_vendidos()
        self.get_produtos()
        
        return self.criar_dataframe()
        
    def get_soup(self, link):
        '''
            Uso da biblioteca requests para GET das informações de cada link
            encontrado no site da variável "self.site", retorna a variável 
            chamada "source".
            
            Uso do BeautifulSoup para o HTML.text que foi armazenado na variável 
            source para cada link encontrado.           
        '''
        
        source = requests.get(link).text
        soup = BeautifulSoup(source, 'html.parser')
        
        return soup
        
    
    def get_mais_vendidos(self):
        '''
            Busca dos links para as categorias mais vendidas no site.
                Return 
                    A variável chamada "self.gp_produtos", onde contém o link
                    dos grupos de produtos.
        '''
        print('Buscando link das categorias mais vendidas')
        
        soup = self.get_soup(self.site)

        for lnk in soup.find_all('section'):
            if lnk.find('a')['href'] not in self.gp_produtos:
                self.gp_produtos.append(lnk.find('a')['href'])
            
    
    def get_produtos(self):
        '''
            Busca dos produtos armazenado na variável "self.gp_produtos".
                Return
                    A variável chamada "self.link_produtos", onde contém todos
                    os links dos produtos mais vendidos por grupo de produto.
        '''
        print('Buscando os produtos selecionados')
        
        for gp in self.gp_produtos[1:]:

            soup = self.get_soup(gp)

            categoria = [cat.find('h1').text.split('Mais vendidos ')[-1] for cat in soup.find_all('aside')][0]

            for pdt in soup.find_all('li'):
                class_ = pdt.get('class', 'Vazio')
                if 'promotion-item' in class_:
                    self.link_produtos.append((categoria, pdt.find('a')['href']))
    
    def criar_dataframe(self):
        '''
            Criar o DataFrame com dos os produtos agrupados por categoria.
                Return 
                    DataFrame com os produtos
        '''

        for pdt in tqdm(self.link_produtos, desc='Criando o DataFrame com os Produtos'):

            soup = self.get_soup(pdt[1])

            categoria = pdt[0]
            titulo = [t.find('h1').text for t in soup.find_all('div',"ui-pdp-header__title-container")][0]
            link = pdt[1]

            try:
                preco = soup.find_all('span', 'andes-money-amount__fraction')[0].text
            except:
                preco = '0.00'

            self.df_list.append(pd.DataFrame(
                [[categoria, titulo , preco, link]], 
                columns=self.columns
                ))
            
        df = pd.concat(self.df_list).reset_index(drop=True)
            
        return df

In [3]:
ml = MercadoLivre()
df = ml.run()

def make_clickable(val):
    '''
        Cria uma coluna chamada "Link Produto", com o formato de acesso ao link.
    '''
    return '<a target="_blank" href="{}">{}</a>'.format(val,'Link Produdo')



'''
 - Será necessário atribuir o DataFrame ao método ".style" seguido pelo ".format"
   com os parâmetros {'nome_coluna': método criado <make_clickable>}

 - Uso do to_excel para salvar o DataFrame criado no seu repositório utilizado. 
'''
df_link = df.style.format({'link': make_clickable})
df_link.to_excel('produtos_mais_vendido_ml.xlsx', index=False)

Buscando link das categorias mais vendidas
Buscando os produtos selecionados


Criando o DataFrame com os Produtos: 100%|██████████| 200/200 [03:57<00:00,  1.19s/it]


In [4]:
'''
    Exemplo de Filtro no DataFrame criado e atribuindo ao mesmo o método ".style"
    seguido pelo ".format" com os parâmetros {'nome_coluna': método criado <make_clickable>}
'''
df.query('categoria == "Eletrodomésticos"').style.format({'link': make_clickable})

,categoria,titulo,preco,link
20,Eletrodomésticos,"Ventilador de mesa Mondial VSP-30 turbo preto com 6 pás cor prata, 30 cm de diâmetro 127 V",169,Link Produdo
21,Eletrodomésticos,Liquidificador portátil Mondial Turbo Power L-99 2.2 L preto com jarra de san 127V,195,Link Produdo
22,Eletrodomésticos,"Ventilador de mesa Mondial VTX-40-8P turbo preto com 8 pás cor prata, 40 cm de diâmetro 127 V",266,Link Produdo
23,Eletrodomésticos,"Ventilador de teto Ventisol Fênix branco com 3 pás de plástico, 96 cm de diâmetro 127 V",192,Link Produdo
24,Eletrodomésticos,"Ventilador de mesa Mondial VSP-30 turbo preto com 6 pás cor prata, 30 cm de diâmetro 220 V",193,Link Produdo
25,Eletrodomésticos,"Ventilador de parede Ventisol New new preto com 3 pás cor preto de plástico, 60 cm de diâmetro 127 V",209,Link Produdo
26,Eletrodomésticos,Liquidificador portátil Mondial Turbo Power L-99 2.2 L preto com jarra de san 220V,140,Link Produdo
27,Eletrodomésticos,"Ventilador de parede Ventisol New new preto com 3 pás cor preto de plástico, 60 cm de diâmetro 220 V",188,Link Produdo
28,Eletrodomésticos,Ferro de passar roupas Black+Decker VFA cor preto e prateado 127V,122,Link Produdo
29,Eletrodomésticos,Liquidificador portátil Mondial Turbo Power L-99 2.2 L vermelho com jarra de san 127V,134,Link Produdo
